## Machine Learning 프로젝트 수행을 위한 코드 구조화

`(분류, 회귀 Task)`

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [ ]:
''' 1. PUBG

(1) killPlace leakage 언급 필요 -> leakage를 찾은 것도 인사이트다.

(2) column 제외 or 다 포함?? -> 둘 다 시도해봐라

(3) 이상치 : train set에만 있으면 제거가 맞지만, test set에 있으면 문제가 있을 수 있다. -> test set도 탐색 필요...
     이상치 데이터를 사용하다보면 overfitting이 있을 수 있다. 
     위 2개의 상황 모두 고려하자

(4) 성능보다는 근거를 바탕으로 모델을 만드는 일련의 과정이 더 중요 ( 끊임없이 '왜 이렇게 나오지?' 를 스스로에게 질문하자)

sklearn.feature_selection.SequentialFeatureSelector : feature의 중요성을 판단
주소 : https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html '''

''' 1. 결측치
- 평균 +/- 분산 -> 범위안에서 랜덤으로 선택
- 0 또는 지정된 숫자로 nan값을 채울 수가 있음 -> -999로 채우는 예시
	- 예) 전체 컬럼의 값이 0, 1, nan입니다. nan = -999로 채우면...
	 -> encoding: 0 -> 1 0 0 , 1 -> 0 1 0 ...
- 해당 컬럼의 이전값 또는 이후값으로 nan값을 채울 수 있음
- EDA를 통해서도 컬럼값을 채울 수 있음

2. feature 수(축소)
- 리소스 부족: 컴퓨터 사양
- 왜 feature 수를 줄여야하는가! -> 다중공선성 문제... 등등
- vif, correlation(pearson) : 1차적으로 줄일 수 있음
- random forest: feature_importances_ -> 각 feature마다 점수가 나온다! 상위 20개 100개
- svm: coef_
- sequential forward/backward selection : 
모델이 학습하면서 feature 선택
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html '''


In [ ]:
''' #네이버크롤링

import requests
from bs4 import BeautifulSoup

url = "https://shopping.naver.com/home/p/index.naver"

resp = requests.get(url)

html = BeautifulSoup(resp.text, 'html.parser')

# 네이버 카테고리 저장
naver_cat = [i.text.strip() for i in html.select("button.button")[:-2]]
print(naver_cat, sep=',')

html = BeautifulSoup(resp.text, 'html.parser')

# 네이버 카테고리 저장
naver_cat = [i.text.strip() for i in html.select("button.button")]
naver_cat '''

In [ ]:
''' #한글이 깨질때 사용하는 코드
import matplotlib.font_manager as fm
# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
font_list
#폰트설치할 위치 알려주는 코드
import matplotlib as mpl
print(mpl.matplotlib_fname())
#fontlist-v330.json 파일 삭제하는 위치알려주는 코드
print(mpl.get_cachedir()) '''

In [ ]:
''' # save traind model by pickle
import pickle
with open('PUBG_randomforest_model.pickle', 'wb') as fw:
    pickle.dump(model, fw)
# load traind model by pickle
with open("datas/advertising.pkl", "rb") as file:
    datas = pickle.load(file) '''

In [1]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [2]:
# flag setting
data_reducing = False ## memory reducing technique
feature_reducing = False ## feature extraction (curse of dimensionality)

In [ ]:
#숨겨진 컬림 보고싶을때
pd.set_option('display.max_columns', None)

In [4]:
# 데이터를 불러옵니다.
train =pd.read_csv('train_V2.csv')
test = pd.read_csv('test_V2.csv')
submission = pd.read_csv('sample_submission_V2.csv')

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [ ]:
## 시각화자료
data.columns.value_counts()
sns.countplot()
sns.histplot()
sns.distplot() #없어질것이니 histplot를 사용해야함
g = sns.catplot(x="day", y="total_bill", hue="smoker", kind="box", data=tips)
#kind="boxen","violin","bar","count" (dodge=False 누적그래프를 보여줌), (그래프방향 바꾸기 edgecolor=".5")
g = sns.catplot(x="day", y="total_bill", hue="sex", kind="violin", split=True, inner="stick", data=tips) # hue를 하나로 합쳐서 그림 split=True
g = sns.catplot(x="class", y="survived", hue="sex",
    palette={"male": "g", "female": "m"},
    markers=["^", "o"], linestyles=["-", "--"],
    kind="point", data=titanic)
#시각화 교육자료 C:\Users\Home\Desktop\이어드림\인공지능머신러닝\Day02-전처리,시각화\03-seaborn_plotting_with_categorical_data.ipynb
sns.scatterplot(x="total_bill", y="tip", hue="smoker", style="smoker", data=tips) #style 은 마커모양을 다르게 표시함
sns.scatterplot(x="total_bill", y="tip", size="size", sizes=(15, 200), alpha=.3, data=tips) #alpha는 투명도 0~1사이값
#시각화 교육자료 C:\Users\Home\Desktop\이어드림\인공지능머신러닝\Day02-전처리,시각화\02-seaborn_statistical_relationship_and_basic.ipynb
sns.barplot(category_amount_sum.index , category_amount_sum["transaction_amount"])
#파이그래프 원형
plt.pie(category_amount_sum.groupby("classification_category").sum()["transaction_amount"], labels=category_amount_sum.groupby("classification_category").sum()["transaction_amount"].index,autopct='%.1f%%')
plt.title("classification_category_sum")

#여러개그래프나타낼때와 인덱스정렬
plt.figure(figsize=(20,20))
n=1
for i in range(4,10):
  plt.subplot(3,3,n)
  df_ver3.loc[i].reindex(['MON','TUE','WED','THU','FRI','SAT' ,'SUN'])
  plt.title(f"Sales by day of the week in {i}")
  sns.barplot(x=df_ver3.loc[i].reindex(['MON','TUE','WED','THU','FRI','SAT' ,'SUN']).index, y=df_ver3.loc[i]["transaction_amount"] )
  n+=1
plt.show() 


#그래프 중간에 색깔바꾸기
plt.figure(figsize=(30,30))
n=1
m=0
z=['red', 'blue', 'yellow', 'purple','red', 'blue', 'yellow', 'purple']
threshold=["2021-03-29","2021-04-17","2021-05-17", "2021-06-17","2021-07-17","2021-08-17","2021-09-17"]
for i in range(3,10) :

  plt.subplot(3,3,n)
  plt.title(f"{i}월 매출 추이", fontsize=15)
  plt.plot(test1[test1["month"]==i]["date_day"], test1[test1["month"]==i]["transaction_amount"], color='grey', label="transaction_amount")
  below_threshold = test1[test1["month"]==i]["date_day"] < threshold[m]
  above_threshold = np.logical_not(below_threshold) # true/false 반대로
  plt.plot(test1[test1["month"]==i]["date_day"][above_threshold], test1[test1["month"]==i]["transaction_amount"][above_threshold], color=z[m])
  
  plt.grid()
  plt.legend(fontsize=13)
  plt.xticks(rotation=90)
  n+=1
  m+=1
plt.show()
#graph_color_change.png 참고






In [ ]:
train.columns.value_counts()

In [ ]:
train.info()

In [ ]:
train.head()

이런 식으로 여러가지 그래프를 그려가며, 데이터에 대한 인사이트를 얻습니다!

### 3. 전처리

#### 결측치 처리

In [7]:
# 결측치가 있는 column
train.isna().sum().sum()
train[train.isnull().any(axis=1)]

1

In [ ]:
train.fillna(0)
titanic['Embarked'].fillna('S',inplace=True)    

In [ ]:
x = [1, 2, 5, 5.5, 5.5, 5.5, 5.5, 6, 9, 10] 
sns.distpxot(x)

In [ ]:
std = np.std(x)    # np를 이용해 std 계산, 약 2.8 
mean = np.mean(x)  # np를 이용한 mean 게산, 약 5.6 

rand_value = np.random.uniform(mean - std, mean + std, 4) # mean 에서 std 만큼 떨어져 있는 랜덤한 값들을 size 만큼 반환

x.extend(rand_value)    # 리스트를 더해줍니다

sns.distpxot(x)

In [ ]:
mean = titanic['Age'].mean()       # age 의 mean 값을 구함
std = titanic['Age'].std()         # age 의 std 값을 구함
size = titanic['Age'].isna().sum() # age 에 nan 값이 몇개인지를 구함 


rand_age = np.random.randint(mean - std, mean + std, size = size) # mean 에서 std 만큼 떨어져 있는 랜덤한 값들을 size 만큼 반환

print(rand_age)

In [ ]:
# 기존 'age'열 중 비어있는 값에 rand_age에 들어있는 값을 차례로 넣기
titanic['Age'].loc[titanic['Age'].isna()] = rand_age

# 아직도 'age'에 nan 값이 있을까요? 확인해봅시다!
titanic.isna().sum()

In [ ]:
for data in titanic:
      titanic['Title'] = titanic['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)    
titanic    # title이라고 새로 생기는 열을 주목해봅시다!
''' > `str.extract()` : 문자추출. 판다스 내장함수. 열 안에서 ()안 조건에 부합하는 string을 추출  
> `' ([A-Za-z]+)\.'` : 기존 string 중에서 ' '(스페이스바)로 시작해 '.'로 끝나는  
하나 이상의 알파벳(대문자 A 부터 소문자 z 까지)을 가져오는 정규표현식입니다  '''

In [ ]:
pd.pivot_table(titanic, values='Survived', columns='Title')

In [ ]:
titanic['solo'] = titanic['SibSp'] + titanic['Parch'] == 0    # 혼자이면 True를 반환, 누군가와 함께 왔으면 False를 반환
titanic['solo'] = titanic['solo'].astype(int)    # .astype()은 데이터셋 또는 데이터셋의 특정 열의 데이터 타입을 바꿔줍니다
titanic.head()                                   # int는 파이썬에서 정수 타입을 의미, float과 반대

In [ ]:
train.describe()

In [11]:
train.describe(include=['O'])

,Id,groupId,matchId,matchType
count,4446966,4446966,4446966,4446966
unique,4446966,2026745,47965,16
top,7f96b2f878858a,14d6b54cdec6bc,4b5db40aec4797,squad-fpp
freq,1,74,100,1756186


In [ ]:
test.info()
test.isna().sum()

In [ ]:
# 중복정보가 있는 column 제거하기 위해 상관계수를 확인해봅니다.
correlated_features = 

In [ ]:
data2 = train.groupId.value_counts()
plt.figure(figsize=(10,8))
sns.histplot(data = data2, bins=100)
plt.show()

In [ ]:
train.corr()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(train.corr(), annot=True, fmt='.2f')
plt.show()
big_five = train.corr().nlargest(5, 'winPlacePerc')['winPlacePerc'].index
big_five

In [ ]:
# outlier 탐색
plt.figure(figsize=(10,10))
sns.boxplot(data=train.walkDistance)
plt.show()

In [ ]:
# outlier 탐색
train.loc[train.walkDistance > 10000].sort_values(by="walkDistance", ascending=False).T

In [ ]:
temp = train.copy()

temp["rankPoints"] = train.loc[:, "rankPoints"].replace(-1, 0)

plt.figure(figsize=(20, 20))
sns.heatmap(temp.corr(), annot=True)
big_five = temp.corr().nlargest(6, 'winPlacePerc')['winPlacePerc'].index
big_five

In [ ]:
#replace 활용
titanic['Title'] = titanic['Title'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer', 'Lady','Major', 'Rev', 'Sir'], 'Other')

In [31]:
## matchtype을 랭크, 노말, 이벤트로 나누기 랭크

def UPPER_matchType(x):
    if x == 'squad-fpp' or x == 'duo' or x == 'solo-fpp'or x == 'squad' or x == 'duo-fpp' or x=='solo':
        return 'Rank'

    elif x == 'normal-solo-fpp' or x == 'normal-duo-fpp' or x == 'normal-squad-fpp' or x == 'normal-duo' or x == 'normal-squad' or x == 'normal-solo':
        return 'Normal'

    else:
        return 'Event'

train['UPPER_matchType'] = train['matchType'].map(UPPER_matchType)

In [10]:
def fpp_tpp(x):
    if x == 'squad-fpp' or x == 'normal-squad-fpp' or x== 'normal-solo-fpp' or x == 'solo-fpp' or x == 'duo-fpp' or x == 'normal-duo-fpp'or x == 'crashfpp' or x == 'flarefpp':
        return 'fpp'

    elif x == 'duo'  or x == 'normal-duo' or x == 'normal-squad' or x == 'normal-solo' or x == 'squad'  or x=='solo' or x == 'crashtpp' or x == 'flaretpp':
        return 'tpp'

train['fpp_tpp'] = train['matchType'].map(fpp_tpp)
train.fpp_tpp.value_counts()

fpp    3320989
tpp    1125977
Name: fpp_tpp, dtype: int64

In [30]:
train.loc[train.fpp_tpp.values == 'tpp','winPlacePerc']
print(train[train.fpp_tpp.values == 'fpp']['winPlacePerc'].mean())
print(train[train.fpp_tpp.values == 'tpp']['winPlacePerc'].mean())

0.47578079863582795
0.4640934702040986


In [28]:
''' #한글 컬럼명 바꾸기
train.columns = ["index", "SEND_SPG_INNB", "REC_SPG_INNB", "CATEGORY", "INVOCE_NUM"]
test.columns = ["index", "SEND_SPG_INNB", "REC_SPG_INNB", "CATEGORY"] '''


#### 다중공선성 처리

In [ ]:
# 상관계수가 threshold(e.g. 0.8)를 넘기는 feature들을 제거합니다.
threshold = 

#### Encoding Categorical Features

In [30]:
# dtype이 object인 변수들 처리.
train.head()
temp = train.copy()

In [31]:
temp

,SEND_SPG_INNB,REC_SPG_INNB,CATEGORY,INVOCE_NUM
0,5011000595017300,2871000192069300,음반,3
1,4148000690043300,5011000264024400,문화컨텐츠,3
2,5011000078068400,1120000007005400,농산물,3
3,4127100048006400,5011000587019400,기타식품,7
4,5011000078068400,2823700010076300,농산물,3
...,...,...,...,...
31679,4471000290087200,5011000213073200,스포츠잡화,3
31680,1129000014045300,5011000319087100,스마트디바이스,4
31681,1129000014045300,5011000263065200,스마트디바이스,6
31682,4127300065073100,5011000264061200,지갑,7


In [44]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
encoder.fit(temp["CATEGORY"])

temp["CATEGORY"]=encoder.transform(temp["CATEGORY"])
test['CATEGORY'] = encoder.transform(test['CATEGORY'])



In [ ]:
# inverse transform 을 이용하면 역으로 원래 문자열 데이터를 알 수 있습니다. 
encoder.inverse_transform([3])

In [ ]:
# map 함수 사용
genders = {"male": 0, "female": 1}
titanic['Sex'] = titanic['Sex'].map(genders)
titanic.head()

In [ ]:
titanic['Embarked'] = titanic['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)    # 소숫점 자리가 나오지 않게 .astype(int)를 사용

In [ ]:

#groupby, merge 사용방법
mean_purchase =df.groupby('User_ID')["Purchase"].mean().rename("User_mean").reset_index() 
df_1 = df.merge(mean_purchase)
#같은 방법으로 transform을 사용할 수 있다.
df["User_Mean"] = df.groupby('User_ID')["Purchase"].transform('mean')



In [47]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
temp_scaled=scaler.fit_transform(temp.drop(["INVOCE_NUM","CATEGORY"], axis=1))



In [48]:
temp_scaled=pd.DataFrame(temp_scaled, columns=["SEND_SPG_INNB",	"REC_SPG_INNB"])

In [49]:
temp[["INVOCE_NUM","CATEGORY"]]

,INVOCE_NUM,CATEGORY
0,3,67
1,3,34
2,3,27
3,7,17
4,3,27
...,...,...
31679,3,55
31680,4,52
31681,6,52
31682,7,79


In [50]:
temp_scaled=pd.concat([temp_scaled, temp[["INVOCE_NUM","CATEGORY"]]],axis=1)

In [51]:
temp_scaled

,SEND_SPG_INNB,REC_SPG_INNB,INVOCE_NUM,CATEGORY
0,0.434214,-0.707699,3,67
1,-0.600409,0.920124,3,34
2,0.434213,-2.039623,3,27
3,-0.625466,0.920124,7,17
4,0.434213,-0.743679,3,27
...,...,...,...,...
31679,-0.213175,0.920124,3,55
31680,-4.219791,0.920124,4,52
31681,-4.219791,0.920124,6,52
31682,-0.625226,0.920124,7,79


In [52]:
train_X = temp_scaled.drop("INVOCE_NUM", axis=1)
train_Y = temp_scaled["INVOCE_NUM"]

In [61]:
train_X

,SEND_SPG_INNB,REC_SPG_INNB,CATEGORY
7540,-0.608802,0.920124,14
1683,0.436611,-0.672709,27
7959,0.434214,0.246174,66
269,0.436612,0.698010,27
8309,-0.600409,0.920124,34
...,...,...,...
8874,0.434213,0.256824,27
23784,0.436612,-0.829406,27
5482,0.436612,-2.031636,27
13338,0.434213,0.387734,27


#### feature extraction

- 차원의 저주를 해결하거나, 데이터의 feature 조합을 이용하는 새로운 feature를 생성할 때, PCA를 사용합니다.

- 분석에 사용할 feature를 선택하는 과정도 포함합니다.

In [ ]:
# PCA 적용
from sklearn.decomposition import PCA

if feature_reducing:
    pca = PCA(n_components=0.9) # PCA(n_components=6)
    pca_data = pca.fit_transform(X)

### 4. 학습 데이터 분할

In [67]:
train_X = temp_scaled.drop("INVOCE_NUM", axis=1)
train_Y = temp_scaled["INVOCE_NUM"]
#모델 정의
model = LGBMRegressor()

# 데이터 스플릿
from sklearn.model_selection import train_test_split
train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size=0.2, random_state=42)

# 데이터 shape
print(f"Train_X: {train_X.shape}  Val_X: {val_X.shape}  Train_Y: {train_Y.shape} || Val_Y: {val_Y.shape}")

# 모델 학습
model.fit(train_X, train_Y)

# 모델 예측
pred_val = model.predict(val_X)

# 모델 평가
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(val_Y, pred_val)))
print("MSE: ",(mean_squared_error(val_Y, pred_val)))

Train_X: (25347, 3)  Val_X: (6337, 3)  Train_Y: (25347,) || Val_Y: (6337,)
5.847586941644184
MSE:  34.19427304008758


In [ ]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
# train : test = 8 : 2
from sklearn.model_selection import train_test_split

X = 
y =

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

### 5. 학습 및 평가

In [ ]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)

param_grid = {
    "max_bin" : 20,
    "learning_rate" : 0.0025,
    "objective" : "regression",
    "boosting_type" : "gbdt",
    "metric" : "mae",
    "sub_feature" : 0.345,
    "bagging_fraction" : 0.85,
    "bagging_freq" : 40,
    "num_leaves" : 512,
    "min_data" : 500,
    "min_hessian" : 0.05,
    "verbose" : 2,
    "feature_fraction_seed" : 2,
    "bagging_seed" : 3
}

model = LGBMRegressor(**param_grid)

In [ ]:
print("\nFitting LightGBM...")
model.fit(X_train, y_train)

In [ ]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = 

In [ ]:
print("Prediction")
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)


train_score = evaluation_metric(y_train, pred_train)
test_score = evaluation_metric(y_test, pred_test)

print("Train Score : %.4f" % train_score)
print("Test Score : %.4f" % test_score)

### 6. Hyper-parameter Tuning

> GridSearchCV

** LightGBM의 hyperparameter **

[Official Documentation] https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html 

[Blog 1] https://smecsm.tistory.com/133

[Blog 2] https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

[Blog 3] https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

In [ ]:
# GridSearchCV를 이용하여 가장 좋은 성능을 가지는 모델을 찾아봅시다. (이것은 첫번째엔 선택입니다.)
# Lightgbm은 hyper-parameter의 영향을 많이 받기 때문에, 저는 보통 맨처음에 한번 정도는 가볍게 GCV를 해봅니다.
# 성능 향상이 별로 없다면, lightgbm으로 돌린 대략적인 성능이 이 정도라고 생각하면 됩니다.
# 만약 성능 향상이 크다면, 지금 데이터는 hyper-parameter tuning을 빡빡하게 하면 성능 향상이 많이 이끌어 낼 수 있습니다.

from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth" : [8, 16, None],
    "n_estimators" : [100, 300, 500],
    "max_bin" : [20],
    "learning_rate" : [0.001, 0.0025, 0.003],
    "objective" : ["regression"],
    "boosting_type" : ["gbdt"],
    "metric" : ["mae"],
    "sub_feature" : [0.345],
    "bagging_fraction" : [0.7, 0.75, 0.85],
    "bagging_freq" : [40],
    "num_leaves" : [256, 512],
    "min_data" : [500],
    "verbose" : [-1], # 필수
    "min_hessian" : [0.05],
    "feature_fraction_seed" : [2],
    "bagging_seed" : [3]
}


gcv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5,
                  n_jobs=-1, verbose=1)

gcv.fit(X_train, y_train)
print("Best Estimator : ", gcv.best_estimator_)

In [ ]:
print("Prediction with Best Estimator")
gcv_pred_train = gcv.predict(X_train)
gcv_pred_test = gcv.predict(x_test)

gcv_train_score = evaluation_metric(y_train, gcv_pred_train)
gcv_test_score = evaluation_metric(y_test, gcv_pred_test)

print("Train MAE Score : %.4f" % gcv_train_score)
print("Test MAE Score : %.4f" % gcv_test_score)

In [ ]:
print("Performance Gain") # 이걸로 성능 향상 확인.
print("in train : ", (train_score - gcv_train_score))
print("in test : ", (test_score - gcv_test_score))

> optuna를 사용해봅시다 !

In [ ]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    n_estimators = 
    max_depth = 
    max_features = 
    
    
    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = RandomForestRegressor(n_estimators=n_estimators,
                                 max_depth=max_depth,
                                 max_features=max_features)
    
    
    # K-Fold Cross validation을 구현합니다.
    folds = KFold(n_splits=K)
    losses = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = mean_absolute_error(y_val, preds)
        losses.append(loss)
    
    
    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [ ]:
K = # Kfold 수
opt_func = partial(optimizer, X=X_train, y=y_train, K)

study = optuna.create_study(direction="minimize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(opt_func, n_trials=5)

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

In [ ]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

In [ ]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

In [ ]:
model = RandomForestRegressor(n_estimators=study.best_trial.params["n_estimators"],
                                 max_depth=study.best_trial.params["max_depth"],
                                 max_features=study.best_trial.params["max_features"])

model.fit(X_train, y_train)
preds = model.predict(X_test)
preds

In [70]:
test

,SEND_SPG_INNB,REC_SPG_INNB,CATEGORY
0,4167000577042200,5011000435014100,47
1,1156000009012200,5011000172034400,12
2,4122000363057300,5011000361097300,88
3,5011000436041400,2826000084036400,58
4,4150000241065200,5011000169044300,39
...,...,...,...
7915,5011000266051200,4623000417038100,27
7916,1154500001098300,5011000264055100,34
7917,5013000610049100,1147000018091400,27
7918,5013000610049100,3117000039026100,27


In [68]:
X_test # 원본 데이터랑 id가 맞는지 확인 해보기!

NameError: name 'X_test' is not defined

In [81]:
test_scaled=scaler.fit_transform(test.drop("CATEGORY", axis=1))

In [84]:
test.columns

Index(['SEND_SPG_INNB', 'REC_SPG_INNB', 'CATEGORY'], dtype='object')

In [87]:
test_scaled=pd.DataFrame(test_scaled, columns=['SEND_SPG_INNB', 'REC_SPG_INNB'])

In [90]:
test_scaled=pd.concat([test_scaled, test["CATEGORY"]], axis=1)

In [72]:
test

,SEND_SPG_INNB,REC_SPG_INNB,CATEGORY
0,4167000577042200,5011000435014100,47
1,1156000009012200,5011000172034400,12
2,4122000363057300,5011000361097300,88
3,5011000436041400,2826000084036400,58
4,4150000241065200,5011000169044300,39
...,...,...,...
7915,5011000266051200,4623000417038100,27
7916,1154500001098300,5011000264055100,34
7917,5013000610049100,1147000018091400,27
7918,5013000610049100,3117000039026100,27


In [91]:
pred=model.predict(test_scaled)

In [92]:
pred

array([4.40098435, 3.20390943, 4.83183857, ..., 4.31160126, 4.43034911,
       2.99292668])

In [93]:
submission.to_csv('baseline.csv',index = False)

In [94]:
df = pd.read_csv('sample_submission.csv')

In [95]:
df["운송장_건수"] = pred

In [96]:
df.to_csv('baseline.csv', index=False)

In [97]:
df

,index,운송장_건수
0,0,4.400984
1,1,3.203909
2,2,4.831839
3,3,5.040676
4,4,4.184698
...,...,...
7915,7915,4.613010
7916,7916,2.992927
7917,7917,4.311601
7918,7918,4.430349


In [ ]:
submission = pd.DataFrame() # submission을 생성합니다.
submission

In [ ]:
submission.reset_index(drop=True).to_csv("submission.csv", index=False)